# Cluster the neighborhoods in Toronto
## In this section we will Use the Notebook to build the code to scrape the following Wikipedia page

Import necessary packages.

In [33]:
import pandas as pd 
import numpy as np


In [34]:
!pip install html5lib
!pip install lxml
!pip install beautifulSoup4
!pip install folium

Scrape the Wikipedia page.

In [35]:
data=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
data.head(5)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove "Not assigned" at the end of each string in the Neighborhood column

In [36]:
data = data.drop(data[(data.Borough == "Not assigned")].index)
data.Neighbourhood.replace("Not assigned", data.Borough, inplace=True)
data.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [37]:
data.Neighbourhood.fillna(data.Borough, inplace=True)
data=data.drop_duplicates()
data.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Group all neighborhoods with the same postal code

In [38]:
data = data.groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(", ".join).reset_index()
data.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [39]:
print("Data shape :",data.shape)

Data shape : (103, 3)


# 3. Get the latitude and the longitude coordinates of each neighborhood.¶


In [40]:
data_geo = pd.read_csv("https://cocl.us/Geospatial_data")
data_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We need to merge 2 dataframes "data" and "df_geo" into one dataframe.

In [41]:
newdata=pd.merge(data,data_geo)
newdata.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Getting all the rows from the data frame which contains Toronto in their Borough.


In [42]:
data_toronto = newdata[newdata['Borough'].str.contains('Toronto',regex=False)]
data_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Visualizing all the Neighbourhoods of the above data frame using Folium

In [43]:
import folium 

map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(data_toronto['Latitude'],data_toronto['Longitude'],data_toronto['Borough'],data_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto


Using KMeans clustering for the clsutering of the neighbourhoods


In [49]:
from sklearn.cluster import KMeans
k=5
toronto_clustering = data_toronto.drop(['Postal Code','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_

array([4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 3, 1, 1, 1, 2, 2, 2, 1, 0, 1, 1, 0, 0, 0, 2, 4], dtype=int32)

In [50]:
data_toronto.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,4,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,4,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,4,M4M,East Toronto,Studio District,43.659526,-79.340923
44,2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [53]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(data_toronto['Latitude'],data_toronto['Longitude'], data_toronto['Neighbourhood'], data_toronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
